In [3]:
import numpy as np

counts = 0

In [90]:
def traditional_multiply_matrix(A, B):
    result = np.zeros((len(A), len(B)))
    global counts
    s = len(A)

    for i in range(s):
        for j in range(s):
            sum_j = 0
            for k in range(s):

                sum_j += A[i][k] * B[k][j]
                counts += 1

            result[i][j] = sum_j

    return result

In [89]:
def multiply_matrix(A, B, l):
    try:
        if A.shape[1] != B.shape[0]:
            raise ValueError(
                "Liczba kolumn pierwszej macierzy musi być równa liczbie wierszy drugiej macierzy"
            )

        if (
            A.shape[0] < 2**l
            and A.shape[1] < 2**l
            and B.shape[0] < 2**l
            and B.shape[1] < 2**l
        ):
            return traditional_multiply_matrix(A, B)
    except ValueError as e:
        print(f"Błąd: {e}")

In [88]:
def add_matrices(matrix1, matrix2):
    global counts
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        raise ValueError("Matrices must have the same dimensions")

    result = [[0 for _ in range(len(matrix1[0]))] for _ in range(len(matrix1))]

    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            result[i][j] = matrix1[i][j] + matrix2[i][j]
            counts += 1

    return result

In [87]:
def subtract_matrices(matrix1, matrix2):
    global counts
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        raise ValueError("Matrices must have the same dimensions")

    result = [[0 for _ in range(len(matrix1[0]))] for _ in range(len(matrix1))]

    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            result[i][j] = matrix1[i][j] - matrix2[i][j]
            counts += 1

    return result

In [91]:
def split_matrix(matrix):
    n = len(matrix) // 2

    return (
        np.array(matrix)[:n, :n],
        np.array(matrix)[:n, n:],
        np.array(matrix)[n:, :n],
        np.array(matrix)[n:, n:],
    )


def strassen_multiply_matrix(A, B, l):

    if len(A) <= 2**l:

        return traditional_multiply_matrix(A, B)

    A11, A12, A21, A22 = split_matrix(A)
    B11, B12, B21, B22 = split_matrix(B)

    P1 = strassen_multiply_matrix(add_matrices(A11, A22), add_matrices(B11, B22), l)
    P2 = strassen_multiply_matrix(add_matrices(A21, A22), B11, l)
    P3 = strassen_multiply_matrix(A11, subtract_matrices(B12, B22), l)
    P4 = strassen_multiply_matrix(A22, subtract_matrices(B21, B11), l)
    P5 = strassen_multiply_matrix(add_matrices(A11, A12), B22, l)
    P6 = strassen_multiply_matrix(
        subtract_matrices(A21, A11), add_matrices(B11, B12), l
    )
    P7 = strassen_multiply_matrix(
        subtract_matrices(A12, A22), add_matrices(B21, B22), l
    )

    C11 = add_matrices(subtract_matrices(add_matrices(P1, P4), P5), P7)
    C12 = add_matrices(P3, P5)
    C21 = add_matrices(P2, P4)
    C22 = add_matrices(subtract_matrices(add_matrices(P1, P3), P2), P6)

    C = np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))

    return C

In [97]:
import datetime

l = [2, 3, 5, 7]
for i in l:
    times = []
    for k in range(2, 8):

        start = datetime.datetime.now()
        strassen_multiply_matrix(
            np.random.randint(10, size=(2**k, 2**k)),
            np.random.randint(10, size=(2**k, 2**k)),
            i,
        )

        times.append((datetime.datetime.now() - start).total_seconds())
        with open(f"times{i}.txt", "w+") as f:
            for items in times:
                f.write("%s\n" % items)

In [ ]:
for i in l:
    table = []
    times = []
    for k in range(2, 8):

        counts = 0
        start = datetime.datetime.now()
        strassen_multiply_matrix(
            np.random.randint(10, size=(2**k, 2**k)),
            np.random.randint(10, size=(2**k, 2**k)),
            i,
        )

        times.append((datetime.datetime.now() - start).total_seconds())
        table.append(counts)

        with open(f"counts{i}.txt", "w+") as f:

            for items in table:

                f.write("%s\n" % items)

        with open(f"times{i}.txt", "w+") as f:
            for items in times:
                f.write("%s\n" % items)

In [59]:
import matplotlib.pyplot as plt


lines = []
with open("counts.txt") as file:
    for line in file:
        for word in line.split():
            lines.append(word)


print(lines)
# plt.xlabel("2^n - size of matrix")
# plt.ylabel("Time difference in seconds")
# plt.plot(list(range(2, 17)), times)
# plt.show()

['96', '48', '105', '684', '342', '742', '4800', '2400', '5201', '33612', '16806', '36414', '235296', '117648', '254905', '1647084', '823542', '1784342']


In [94]:
k = 6
counts = 0

A, B = np.random.randint(10, size=(2**k, 2**k)), np.random.randint(
    10, size=(2**k, 2**k)
)


print(
    traditional_multiply_matrix(
        A,
        B,
    )
)
print(counts)
counts = 0

print(
    strassen_multiply_matrix(
        A,
        B,
        4,
    )
)
print(counts)

[[1384. 1391. 1317. ... 1435. 1351. 1404.]
 [1294. 1266. 1225. ... 1393. 1305. 1382.]
 [1134. 1017. 1317. ... 1263. 1132. 1446.]
 ...
 [1172. 1094. 1195. ... 1164. 1148. 1311.]
 [1343. 1394. 1233. ... 1404. 1163. 1356.]
 [1406. 1348. 1429. ... 1397. 1501. 1517.]]
262144
[[1384. 1391. 1317. ... 1435. 1351. 1404.]
 [1294. 1266. 1225. ... 1393. 1305. 1382.]
 [1134. 1017. 1317. ... 1263. 1132. 1446.]
 ...
 [1172. 1094. 1195. ... 1164. 1148. 1311.]
 [1343. 1394. 1233. ... 1404. 1163. 1356.]
 [1406. 1348. 1429. ... 1397. 1501. 1517.]]
251392


## Tests


In [81]:
A = np.array([[1, 3], [2, 2]])
A

array([[1, 3],
       [2, 2]])

In [82]:
B = np.array([[2, 2], [6, 3]])
B

array([[2, 2],
       [6, 3]])

In [ ]:
counts = 0
print(traditional_multiply_matrix(A, B))
print(counts)
counts = 0


print(strassen_multiply_matrix(A, B))
print(counts)

[[20. 11.]
 [16. 10.]]
8
[[20. 11.]
 [16. 10.]]
8


In [19]:
multiply_matrix(A, B, 2)

array([[20., 11.],
       [16., 10.]])

In [ ]:
C = np.array([[1, 2]])
C

array([[1, 2]])

In [ ]:
multiply_matrix(A, C, 2)

Błąd: Liczba kolumn pierwszej macierzy musi być równa liczbie wierszy drugiej macierzy
